### Import library

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os.path
import re

### Define some function

In [2]:
def scroll_down(driver):
    SCROLL_PAUSE_TIME = 1
    while True:
        last_height = driver.execute_script("return document.body.scrollHeight")
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # try again (can be removed)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            # check if the page height has remained the same
            if new_height == last_height:
                # if so, you are done
                break
            # if not, move on to the next loop
            else:
                last_height = new_height
                continue
    return driver


def parse_info_product(driver, item, label):
    driver.get(item)
    time.sleep(5)
### This part for get item name
    info_dict = {}
    product_span = driver.find_elements_by_class_name("qaNIZv")
    if len(product_span)>0:
        info_dict.update({'Tên sp:': product_span[0].text.split('\n')[-1],'label:': label})
        ### This part for get item description
        for attr in driver.find_elements_by_class_name("kIo6pj"):
            info_dict.update({attr.text.split('\n')[0]:'-'.join(attr.text.split('\n')[1:])})
        ### This part for get price - discount - price discount
        if len(driver.find_elements_by_class_name("_3_ISdg"))==0:
            info_dict.update({'sale_price':driver.find_elements_by_class_name("_3n5NQx")[0].text})
            info_dict.update({'org_price':driver.find_elements_by_class_name("_3n5NQx")[0].text})
            info_dict.update({'discount':0})
        else:
            info_dict.update({'org_price':driver.find_elements_by_class_name("_3_ISdg")[0].text})
            info_dict.update({'sale_price':driver.find_elements_by_class_name("_3n5NQx")[0].text})
            info_dict.update({'discount':driver.find_elements_by_class_name("MITExd")[0].text})
        ### This part for get rating-star-selled
        if len(driver.find_elements_by_class_name("_3Oj5_n"))==0:
            info_dict.update({'star':0})
            info_dict.update({'rating':0})
        else:
            for rat in driver.find_elements_by_class_name("_3Oj5_n"):
                info_dict.update({'star':driver.find_elements_by_class_name("_3Oj5_n")[0].text})
                info_dict.update({'rating':driver.find_elements_by_class_name("_3Oj5_n")[1].text})
        try:
            my_property = driver.find_element_by_class_name('_3ZDC1p').find_element_by_xpath("//div[@class='_2JMB9h _3XaILN']").value_of_css_property("background-image")
            info_dict.update({'img_url': re.split('[()]',my_property)[1]})
        except NoSuchElementException:
            info_dict.update({'img_url': ''})
        info_dict.update({'selled':driver.find_elements_by_class_name("_22sp0A")[0].text})
        ### This part for get info about store
        info_dict.update({driver.find_elements_by_class_name("TuJk3S")[0].text.split('\n')[0]:driver.find_elements_by_class_name("TuJk3S")[0].text.split('\n')[1]})
        info_dict.update({driver.find_elements_by_class_name("TuJk3S")[0].text.split('\n')[2]:driver.find_elements_by_class_name("TuJk3S")[0].text.split('\n')[3]})
        info_dict.update({driver.find_elements_by_class_name("TuJk3S")[1].text.split('\n')[0]:driver.find_elements_by_class_name("TuJk3S")[1].text.split('\n')[1]})
        info_dict.update({driver.find_elements_by_class_name("TuJk3S")[1].text.split('\n')[2]:driver.find_elements_by_class_name("TuJk3S")[1].text.split('\n')[3]})
        info_dict.update({driver.find_elements_by_class_name("TuJk3S")[2].text.split('\n')[0]:driver.find_elements_by_class_name("TuJk3S")[2].text.split('\n')[1]})
        info_dict.update({driver.find_elements_by_class_name("TuJk3S")[2].text.split('\n')[2]:driver.find_elements_by_class_name("TuJk3S")[2].text.split('\n')[3]})
    else:
        pass
    return driver, info_dict


def get_link_product(driver):
    all_items = driver.find_elements_by_xpath('//a[@data-sqe="link"]')
    all_urls = []
    for item in all_items:
        url = item.get_attribute('href')
        all_urls.append(url)
    return driver, all_urls

### Please change param at cell below

In [3]:
index = 0
filename = 'thoi_trang_nam_ratings'
limit_prod = 251

### This part for execute to crawl data, please run each cell and wait 3s before continue

In [4]:
driver = webdriver.Chrome(executable_path='D:/IT/Python/Crawl_data/chromedriver.exe')
driver.get("https://shopee.vn/")
time.sleep(3)

In [5]:
popup_close_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'shopee-popup__close-btn')]")))
if popup_close_button:
    popup_close_button.click()

In [6]:
driver.maximize_window()

In [7]:
url_item = []
category = driver.find_elements_by_class_name('home-category-list__group')
for sub_item in category:
    for ssitem in sub_item.find_elements_by_class_name('home-category-list__category-grid'):
        url_item.append(ssitem.get_attribute('href'))
url_item = url_item[:-1]

In [8]:
url_item

['https://shopee.vn/Th%E1%BB%9Di-Trang-Nam-cat.78',
 'https://shopee.vn/Th%E1%BB%9Di-Trang-N%E1%BB%AF-cat.77',
 'https://shopee.vn/%C4%90i%E1%BB%87n-Tho%E1%BA%A1i-Ph%E1%BB%A5-Ki%E1%BB%87n-cat.84',
 'https://shopee.vn/M%E1%BA%B9-B%C3%A9-cat.163',
 'https://shopee.vn/Thi%E1%BA%BFt-B%E1%BB%8B-%C4%90i%E1%BB%87n-T%E1%BB%AD-cat.2365',
 'https://shopee.vn/Nh%C3%A0-C%E1%BB%ADa-%C4%90%E1%BB%9Di-S%E1%BB%91ng-cat.87',
 'https://shopee.vn/M%C3%A1y-t%C3%ADnh-Laptop-cat.13030',
 'https://shopee.vn/S%E1%BB%A9c-Kh%E1%BB%8Fe-S%E1%BA%AFc-%C4%90%E1%BA%B9p-cat.160',
 'https://shopee.vn/M%C3%A1y-%E1%BA%A3nh-M%C3%A1y-quay-phim-cat.13033',
 'https://shopee.vn/Gi%C3%A0y-D%C3%A9p-N%E1%BB%AF-cat.161',
 'https://shopee.vn/%C4%90%E1%BB%93ng-H%E1%BB%93-cat.9607',
 'https://shopee.vn/T%C3%BAi-V%C3%AD-cat.162',
 'https://shopee.vn/Gi%C3%A0y-D%C3%A9p-Nam-cat.2429',
 'https://shopee.vn/Ph%E1%BB%A5-Ki%E1%BB%87n-Th%E1%BB%9Di-Trang-cat.80',
 'https://shopee.vn/Thi%E1%BA%BFt-B%E1%BB%8B-%C4%90i%E1%BB%87n-Gia-D%E1%BB%A5ng-c

In [9]:
def parse_user_info(list_author, list_rating, list_comment, driver):
    list_user_info = []
    product_span = driver.find_elements_by_class_name("qaNIZv")
    if len(product_span)>0:
        product = product_span[0].text.split('\n')[-1]
    for idx, author in enumerate(list_author):
        user_name = author.text
        stars = list_rating[idx].get_attribute('innerHTML').split().count('enable-background="new')
        comment = list_comment[idx].text
        list_user_info.append({'username':user_name, 'rating':stars, 'comment': comment,'sp':product})
    else:
        pass
    return driver, list_user_info

In [10]:
def scroll_down_and_wait(driver):
    actions = ActionChains(driver)
    for _ in range(4):
        actions.send_keys(Keys.SPACE).perform()
        time.sleep(3)
    return driver

def scroll_up_and_wait(driver):
    actions = ActionChains(driver)
    for _ in range(2):
        actions.send_keys(Keys.PAGE_UP).perform()
        time.sleep(3)
    return driver

In [11]:
driver.get(url_item[0])
driver = scroll_down(driver)
all_items = driver.find_elements_by_xpath('//a[@data-sqe="link"]')
all_urls = []
for item in all_items:
    url = item.get_attribute('href')
    all_urls.append(url)
all_urls

['https://shopee.vn/%C3%81o-S%C6%A1-mi-nam-ch%E1%BA%A5t-l%E1%BB%A5a-d%C3%A0i-tay-l%C3%B4ng-c%C3%B4ng-ng%E1%BB%B1c-c%E1%BB%95-cao-c%E1%BA%A5p-i.304898825.6950852647',
 'https://shopee.vn/%C3%81o-thun-nam-th%E1%BB%83-thao-cotton-l%E1%BA%A1nh-cao-c%E1%BA%A5p-tay-l%E1%BB%A1-from-r%E1%BB%99ng-Xanh-AD03-Phong-C%E1%BA%A3nh-(44-70KG)-i.304898825.4150860324',
 'https://shopee.vn/%C3%81o-Thun-Nam-Th%E1%BB%83-Thao-C%E1%BB%95-Tr%C3%B2n-H%E1%BB%8Da-Ti%E1%BA%BFt-Alaska-Nhi%E1%BB%81u-M%C3%A0u-Cao-C%E1%BA%A5p-Phong-C%C3%A1ch-Tr%E1%BA%BB-Trung-i.304898825.6050853730',
 'https://shopee.vn/Qu%E1%BA%A7n-L%C3%B3t-Nam-Gi%C3%A1-R%E1%BA%BB-%E2%9D%A4%EF%B8%8F-Free-Ship-%E2%9D%A4%EF%B8%8F-Ch%E1%BA%A5t-M%E1%BB%81m-M%E1%BA%A1i-D%C3%A0y-D%E1%BA%B7n-Si%C3%AAu-%C6%AFng-i.272970397.6150441558',
 'https://shopee.vn/%C3%81o-Thun-Nam-Tay-Ng%E1%BA%AFn-LOGO1970-Nhi%E1%BB%81u-M%C3%A0u-Cao-C%E1%BA%A5p-Phong-C%C3%A1ch-Tr%E1%BA%BB-Trung-i.188903465.6042261281',
 'https://shopee.vn/%C3%81o-Thun-Nam-Th%E1%BB%83-Thao-Thun-L%E1%B

In [13]:
print('Shopee returned data:', len(all_urls))

Shopee returned data: 15


In [ ]:
# for category in url_item:
user_info_all = pd.DataFrame([], columns=['username', 'rating', 'comment', 'sp'])
driver.get(url_item[index])
driver = scroll_down(driver)
driver, all_urls = get_link_product(driver)
print('Shopee returned data:', len(all_urls))
for product_url in all_urls:
    df_user_info_old = pd.DataFrame([], columns=['username', 'rating', 'sp'])
    df_user_info = pd.DataFrame()
    driver.get(product_url)
    time.sleep(5)
    driver = scroll_down_and_wait(driver)
    try:
        while True:
            list_author = driver.find_elements_by_class_name('shopee-product-rating__author-name')
            list_rating = driver.find_elements_by_class_name('shopee-product-rating__rating')
            list_comment = driver.find_elements_by_class_name('shopee-product-rating__content')
            driver, list_user_info = parse_user_info(list_author, list_rating, list_comment, driver)
            if len(list_user_info)==0: 
                break
            else:
                if df_user_info_old.equals(df_user_info):
                    break
                else:
                    df_user_info_old = df_user_info.copy()
                    df_user_info = pd.DataFrame(list_user_info)
                    user_info_all = user_info_all.append(df_user_info)
                    user_info_all = user_info_all.drop_duplicates().reset_index(drop=True)
                    print('User crawled:', user_info_all.shape[0])
                    driver.find_element_by_xpath("//button[@class='shopee-icon-button shopee-icon-button--right ']").click()
                    time.sleep(3)
    except NoSuchElementException:
        continue
    if user_info_all.shape[0]>501:
        break

In [ ]:
print(user_info_all.shape)
user_info_all.to_csv('{}.csv'.format(filename), index=False)

In [ ]:
all_urls